# AIOK - Democratized Neural Architecture Search (DE-NAS)

Neural Architecture Search (NAS) is quickly becoming the standard methodology to design neural network models. However, NAS is typically compute-intensive because multiple models need to be evaluated before choosing the best one. As one of the core capabilities of e2eAIOK, DE-NAS is a hardware-aware train-free neural architecture search framework to generate compact networks, and thus generate lightweight network architectures that can deliver higher throuhgput. DE-NAS is a compact neural architecture search solution that support CV, NLP, ASR and other domains.

DE-NAS core componements include:
- Multi-Model Unified Search Space: DE-NAS supports search spaces including Compute Vision (CV), Natural Language Processing (NLP), and Automatic Speech Recognition (ASR). Meanwhile, the search space utilizes Intel optimized neural operators, and incorporates prior knowledge. Another key innovation is that it is the first of its kind to share a unified transformer-based search space among different models. We leveraged a unified transformer structure for transformer-based search space for CV, NLP and ASR domains.
   
- Hardware-Aware Unified Search Framework: DE-NAS provides hardware-aware unifed search framework, and comes with three types of built-in search strategies: Random Search/ Evolutionary Search/ Bayesian Search, which can cover popular domains of CV/NLP/ASR and enhance DE-NAS with SigOpt-based Bayesian search strategy as an advanced feature.
    
- Train-Free Evaluation: DE-NAS provides a novel zero-cost metric called DE-Score, which combines network expressivity, network complexity, network diversity, network saliecny and network latency. Computation of DE-Score only takes a few forward inferences without training, making it extremely fast, lightweight and data-free. 

<img src="./img/denas_framework.jpg" width="100%">
<center>DE-NAS Architecture</center>

## Multi-Model Unified Search Space

### CNN-based Search Space (covering CV domain)
    
- ResNet like skeleton: consists of residual blocks and bottleneck blocks, number of layers, convolutional layer kernel size, number of filters,

<img src="./img/denas_cnn.png" width="50%">
<center>CNN-based Search Space</center>

### Transformer-based search space (covering CV/NLP/ASR domains)

- Embedding dimension, number of heads, query/key/value dimension, MLP ratio, network depth.
    
<img src="./img/denas_autoformer.gif" width="50%">  
<center>Transformer-based Search Space</center>

## Hardware-Aware Unified Search Framework

DE-NAS employs a unified search framework for different search strategies. It uses factory design pattern in the DE-NAS search framework, which abstracts the domain-independent search process, and provides three built-in search strategies: Random Search/Evolutionary Search/Bayesian Search that can be easily turned on/off with a switch. Bayesian search algorithm was implemented by leveraging SigOpt API.

<img src="./img/denas_search_framework.jpg" width="50%">
<center>DE-NAS Unified Search Framework</center>

By encapsulating search strategies with factory design pattern, users only need to call the unified API searcher.search() and searcher.get_best_structures() to get the optimal neural network structure, and do not need to know the details of search algorithm to find the optimal structure for task domain. Users only need to pass in the search engine type as a parameter, and they can easily switch to different NAS search strategy. This pluggable framework makes it simple for extension. If the user wants to implement new search strategies, they only need to implement the searcher.search() interface. And if the user wants to expand DE-NAS to new domain, they only need to pass super network into the constructor of searcher. Eventually, DE-NAS search framework makes it easy for data scientists to perform NAS on commodity CPU hardware.

### Random Search
Random Search is the basic search strategy for DE-NAS, which maintains a fixed-size max-heap to store the randomly generated structures. For each randomly generated structure, we compute the hardware-aware train-free score and insert it into the heap. After a given number of iterations, the top of heap is returned as optimal structure.

### Evolutionary Search

<img src="./img/denas_ea.png" width="30%">
<center>Evolutionary Search</center>

- Initialization: Initialize super network architecture with search space, and initialize population pool for neural network architectures.
- Selection: In the selection step, the population pool is sorted according to the calculated hardware-aware train-free score and a new fixed-size population pool is generated.
- Crossover: In the crossover step, two network architecture sequences are selected from the population pool and perform cross over to generate new sequences.
- Mutation: In the mutation step, network architecture sequences of the population pool are randomly flipped.

### SigOpt-based Bayesian Search

<img src="./img/sigopt_based_bayesian_search.jpg" width="50%">
<center>SigOpt-based Bayesian Search</center>

- Step1. Building SigOpt Compatible Search Space: 
Since the parameter dependencies in the SigOpt search space are independent by default. Also, SigOpt supports linear constraints and conditional parameters to tackle with parameter dependencies, we need to convert the original search space with dependencies into the search space supported by SigOpt. See ViT search space conversion as a reference: (1) Since MLP_RATIO and NUM_HEADS are variable-length arrays, and the length is determined by the value of DEPTH, we convert variable-length arrays MLP_RATIO and NUM_HEADS of original search space into fixed-length arrays, which length equals max value of DEPTH. (2) Set the last four elements of fixed-length arrays as conditional parameters.

<img src="./img/denas_vit_search_space_original.jpg" width="25%">
<center>Original Vision Transformer Search Space</center>

<img src="./img/denas_vit_search_space_converted.jpg" width="50%">
<center>SigOpt API compatible Vision Transformer Search Space</center>

- Step2. Receive SigOpt Sample Structure: 
Call DE-NAS function _get_sigopt_suggestion() to receive SigOpt suggested neural network structure. This step corresponds to the sample operation of Bayesian Optimization.

- Step3. Evaluate SigOpt Sample Structure:
Call DE-NAS function cand_islegal() function to determine whether the network structure workable or whether the inference latency within budget.
Call DE-NAS function cand_evaluate() function to evaluate hardware-aware train-free score for suggested structure.
This step corresponds to the evaluation operation of Bayesian Optimization. 

- Step4. Send Metrics and Guide Further Search: 
Call DE-NAS function _set_illegal_observation() to mark network structures that unworkable or exceed budget latency as failure and send to SigOpt.
Call DE-NAS function _set_sigopt_observation() to send train-free score to SigOpt.
This step will guide further search process and trade off exploration and exploitation, corresponding to the augment surrogate function step in Bayesian Optimization.

- Step5. Loop Back Step2 to Perform Iteration: Run through SigOpt search budget and return the optimal neural network structure.

## Train-Free Score

The train-free score uses an innovative, zero-cost “proxy” to predict model accuracy instead of full training and validation. It used a novel zero-cost metric combined Gaussian complexity based on network expressivity, NTK score based on network complexity, nuclear norm score based on network diversity, Synflow score based on network saliency, and latency score. The computation of DE-Score only takes a few forward inferences other than iterative training, making it extremely fast, lightweight, and data-free. The overall DE-Score was calculated as following equation:

$DE_Score=(D_EXP+D_COM+D_DIV+D_SAL )∙D_LAT$

## Try DE-NAS!

* DE-NAS for built-in CV/NLP/ASR search space
    * [CV](computer_vision/DENAS_COMPUTER_VISION_DEMO.ipynb) - DE-NAS for CV domain
    * [NLP](bert/DENAS_BERT_DEMO.ipynb) - DE-NAS for NLP domain
    * [ASR](asr/DENAS_ASR_DEMO.ipynb) - DE-NAS for ASR domain